In [34]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from string import digits 
import string
from textblob import TextBlob


In [2]:
os.getcwd()


'C:\\Users\\apurv\\OneDrive\\Documents\\Projects\\2020\\Data_Science_Competitions\\Kaggle_RealOrNot\\Code'

In [4]:
##Loading dataset
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [5]:
##Looking for missing values
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [6]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [7]:
test.describe()

,id
count,3263.000000
mean,5427.152927
std,3146.427221
min,0.000000
25%,2683.000000
50%,5500.000000
75%,8176.000000
max,10875.000000


In [8]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
#get unique values 
len(train.location.unique())

3342

In [19]:
len(test.location.unique())

1603

In [20]:
test.groupby('location').count()

,id,keyword,text
location,,,
,1,1,1
FLA,1,1,1
Waiheke Island,1,1,1
Manchester/Nantwich,1,1,1
?becky?chloe?,1,1,1
...,...,...,...
"ÌÏT: 41.373061,-71.942237",1,1,1
"ÌÏT: 50.953278,-113.978785",1,1,1
"ÌÏT: 58.193556,-5.334943",1,1,1


In [18]:
train.groupby('location').count()
##Can spot very riduculous values

,id,keyword,text,target
location,,,,
,1,1,1,1
Glasgow,1,1,1,1
"Melbourne, Australia",1,1,1,1
News,1,1,1,1
å_,1,1,1,1
...,...,...,...,...
å_: ?? ÌÑ ? : ?,1,1,1,1
å_å_Los Mina Cityã¢,1,1,1,1
å¡å¡Midwest Û¢Û¢,1,1,1,1


In [21]:
len(train.keyword.unique())

222

In [22]:
len(test.keyword.unique())

222

In [23]:
train.groupby('keyword').count()

,id,location,text,target
keyword,,,,
ablaze,36,29,36,36
accident,35,28,35,35
aftershock,34,25,34,34
airplane%20accident,35,29,35,35
ambulance,38,26,38,38
...,...,...,...,...
wounded,37,27,37,37
wounds,33,27,33,33
wreck,37,27,37,37


In [24]:
train.groupby('keyword').sum()

,id,target
keyword,,
ablaze,2534,13
accident,4263,24
aftershock,5825,0
airplane%20accident,7705,30
ambulance,10224,20
...,...,...
wounded,392538,26
wounds,351859,10
wreck,396215,7


In [25]:
test.groupby('keyword').sum()

,id
keyword,
ablaze,991
accident,1762
aftershock,2700
airplane%20accident,3320
ambulance,3301
...,...
wounded,137937
wounds,181116
wreck,139260


In [30]:
##Cleaning the text tweet

## changing everything to lower case
train['text_clean'] = train.apply(lambda x: x['text'].lower(),axis=1)
test['text_clean'] = test.apply(lambda x: x['text'].lower(),axis=1)

## removing punctuations
train['text_clean'] = train.apply(lambda x: x['text_clean'].translate(str.maketrans('','',string.punctuation)),axis=1)
test['text_clean'] = test.apply(lambda x: x['text_clean'].translate(str.maketrans('','',string.punctuation)),axis=1)

## removing digits
train['text_clean'] = train.apply(lambda x: x['text_clean'].translate(str.maketrans('', '', digits)) ,axis=1)
test['text_clean'] = test.apply(lambda x: x['text_clean'].translate(str.maketrans('', '', digits) ) ,axis=1)

In [36]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\apurv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [37]:
##lemmatize verbs and nouns
## in final model only nouns have been lemmatize, verbs used to indicate polarity
def lemmat_words(words,pos):
    lemmated = [lemmatizer.lemmatize(word,pos) for word in words]
    return lemmated

lemmatizer = WordNetLemmatizer()
train['text_clean_lem'] = train.apply(lambda x: lemmat_words(x['text_clean'],pos = wordnet.NOUN),axis=1)
test['text_clean_lem'] = test.apply(lambda x: lemmat_words(x['text_clean'],pos = wordnet.NOUN),axis=1)


In [38]:
train.head(10)

,id,keyword,location,text,target,text_clean,text_clean_lem
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[o, u, r, , d, e, e, d, s, , a, r, e, , t, ..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[f, o, r, e, s, t, , f, i, r, e, , n, e, a, ..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[a, l, l, , r, e, s, i, d, e, n, t, s, , a, ..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...,"[ , p, e, o, p, l, e, , r, e, c, e, i, v, e, ..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[j, u, s, t, , g, o, t, , s, e, n, t, , t, ..."
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,rockyfire update california hwy closed in bo...,"[r, o, c, k, y, f, i, r, e, , u, p, d, a, t, ..."
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster heavy rain causes flash floodin...,"[f, l, o, o, d, , d, i, s, a, s, t, e, r, , ..."
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,im on top of the hill and i can see a fire in ...,"[i, m, , o, n, , t, o, p, , o, f, , t, h, ..."
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,theres an emergency evacuation happening now i...,"[t, h, e, r, e, s, , a, n, , e, m, e, r, g, ..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,im afraid that the tornado is coming to our area,"[i, m, , a, f, r, a, i, d, , t, h, a, t, , ..."
